In [5]:
import os
import pandas as pd
import os
from soundtype_params import SoundtypeParams
import numpy as np
import xlrd
import pandas as pd

# F:\中建科技项目数据\history\use\2021-12-27\xls\Record109848.xls

root_dir = r"F:\中建科技项目数据\history\use"
file_name = "Record109848" + ".xls"

In [6]:
# 输入根目录和文件名，返回文件路径
def seek_files(root_dir, file_name):
    for root, dirs, files in os.walk(root_dir):
        if file_name in files:
            path = "{0}\\{1}".format(root, file_name)
            path = path.replace('\\', '/')
    return path

In [7]:
path = seek_files(root_dir, file_name)
path

UnboundLocalError: local variable 'path' referenced before assignment

In [ ]:
# 读取麒麟山庄的对照表
qilin_path = r'.\input\2021 新闻大厦 麒麟山庄数据.xlsx'
qilin_name = "麒麟山庄"
qilin_data = pd.read_excel(qilin_path, sheet_name=qilin_name)
# 抽取出leq在50-70之间的
qilin_use_data = qilin_data[(float(50) < qilin_data['Leq']) & (qilin_data['Leq'] < float(70))]
# 重新编号
qilin_use_data.reset_index(drop=True, inplace=True)
qilin_use_data

In [ ]:
# 提取每个文件的特征
def extract_features(path):
    wb = xlrd.open_workbook(path)
    params_dict = {}

    for idx in range(6):
        sheet = wb.sheet_by_index(idx)
        # 读取同一个excel文件中的不同sheet
        print("Sheet", idx + 1)
        param_type = sheet.cell(10, 1).value
        if param_type == 'pressure':
            param_type = 'Leq'
        elif param_type == 'fluctuation strength':
            param_type = 'Fluct'
        else:
            # 不用小写
            # param_type = param_type.lower()
            # 首字母大写
            param_type = param_type.capitalize()
        values = []
        times = []

        for i in range(15, sheet.nrows):
            time = float(sheet.cell(i, 0).value)
            values.append(float(sheet.cell(i, 1).value))
            times.append(time)

        values = np.array(values)
        times = np.array(times)
        mean = values.mean()
        var = values.var()
        std = values.std(ddof=1)
        max = values.max()
        p_10 = np.percentile(values, 10, interpolation='midpoint')
        p_25 = np.percentile(values, 25, interpolation='midpoint')
        p_50 = np.percentile(values, 50, interpolation='midpoint')
        p_75 = np.percentile(values, 75, interpolation='midpoint')
        p_90 = np.percentile(values, 90, interpolation='midpoint')
        p_10_90 = p_90 - p_10

        params_dict["%s_mean" % param_type] = mean
        params_dict["%s_var" % param_type] = var
        params_dict["%s_std" % param_type] = std
        params_dict["%s_max" % param_type] = max
        params_dict["%s_10" % param_type] = p_10
        params_dict["%s_25" % param_type] = p_25
        params_dict["%s_median" % param_type] = p_50
        params_dict["%s_75" % param_type] = p_75
        params_dict["%s_90" % param_type] = p_90
        params_dict["%s_10_90" % param_type] = p_10_90

    soundtype_params = None
    try:
        file_name = os.path.basename(path).replace("xlsx", "wav")
        print(file_name, "is processed !")
        leq_mean = params_dict["Leq_mean"]
        leq_std = params_dict["Leq_std"]
        leq_25 = params_dict["Leq_25"]
        leq_median = params_dict["Leq_median"]
        leq_75 = params_dict["Leq_75"]
        leq_10_90 = params_dict["Leq_10_90"]
        loudness_mean = params_dict["Loudness_mean"]
        loudness_std = params_dict["Loudness_std"]
        loudness_25 = params_dict["Loudness_25"]
        loudness_median = params_dict["Loudness_median"]
        loudness_75 = params_dict["Loudness_75"]
        loudness_10_90 = params_dict["Loudness_10_90"]
        roughness_mean = params_dict["Roughness_mean"]
        roughness_std = params_dict["Roughness_std"]
        roughness_25 = params_dict["Roughness_25"]
        roughness_median = params_dict["Roughness_median"]
        roughness_75 = params_dict["Roughness_75"]
        roughness_10_90 = params_dict["Roughness_10_90"]
        sharpness_mean = params_dict["Sharpness_mean"]
        sharpness_std = params_dict["Sharpness_std"]
        sharpness_25 = params_dict["Sharpness_25"]
        sharpness_median = params_dict["Sharpness_median"]
        sharpness_75 = params_dict["Sharpness_75"]
        sharpness_10_90 = params_dict["Sharpness_10_90"]
        fluct_mean = params_dict["Fluct_mean"]
        fluct_std = params_dict["Fluct_std"]
        fluct_25 = params_dict["Fluct_25"]
        fluct_median = params_dict["Fluct_median"]
        fluct_75 = params_dict["Fluct_75"]
        fluct_10_90 = params_dict["Fluct_10_90"]
        tonality_mean = params_dict["Tonality_mean"]
        tonality_std = params_dict["Tonality_std"]
        tonality_25 = params_dict["Tonality_25"]
        tonality_median = params_dict["Tonality_median"]
        tonality_75 = params_dict["Tonality_75"]
        tonality_10_90 = params_dict["Tonality_10_90"]
        soundtype_params = SoundtypeParams(file_name, leq_mean, leq_std, leq_25, leq_median, leq_75, leq_10_90,
                                           loudness_mean, loudness_std, loudness_25, loudness_median, loudness_75,
                                           loudness_10_90, roughness_mean, roughness_std, roughness_25,
                                           roughness_median, roughness_75, roughness_10_90, sharpness_mean,
                                           sharpness_std, sharpness_25, sharpness_median, sharpness_75, sharpness_10_90,
                                           fluct_mean, fluct_std, fluct_25, fluct_median, fluct_75, fluct_10_90,
                                           tonality_mean, tonality_std, tonality_25, tonality_median, tonality_75,
                                           tonality_10_90)
    except Exception as e:
        print("[ERROR] XlsProcessor: %s" % e)

    params_dict["name"] = os.path.basename(path)
    single_data = pd.DataFrame(params_dict, [0])

    return single_data

In [ ]:
# 测试例子
path = r'F:\中建科技项目数据\history\use\2021-12-27\xls\Record109848.xls'
example = extract_features(path)
columns = example.columns

In [ ]:
# 新建一个dataframe用来存储麒麟山庄的特征
qilin_features = pd.DataFrame(columns=columns)
root_dir = r"F:\中建科技项目数据\history\use"

# 循环执行,只用qilin_use_data
for i in range(len(qilin_use_data)):
    file_name = qilin_use_data.loc[i,"file_name"] + ".xls"
    # 查找路径
    path = seek_files(root_dir, file_name)
    # 提取特征
    single_data = extract_features(path)
    # 记住要 = ！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！
    # 血泪史
    qilin_features = qilin_features.append(single_data,ignore_index=True)

qilin_features

In [ ]:
# test
single_data = extract_features(r'F:\中建科技项目数据\history\use\2021-12-27\xls\Record109848.xls')
single_data
a = pd.DataFrame(columns=columns)
a = a.append(single_data)
a = a.append(single_data)
a = a.append(single_data,ignore_index=True)
a

In [ ]:
write_path = r'.\output\qilin_features.csv'
qilin_features.to_csv(write_path)